In [10]:
import pandas as pd
import numpy as np
import copy, time
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

data_path = 'mid/'
df_train = pd.read_csv(data_path + 'train_data.csv')
df_test = pd.read_csv(data_path + 'test_features.csv')

train_Y = np.log1p(df_train['poi'])
ids = df_test['name']

train_data = df_train.drop(['name','email_address'] , axis=1)
test_features = df_test.drop(['email_address'] , axis=1)
df_train = df_train.drop(['name','poi', 'email_address'] , axis=1)
df_test = df_test.drop(['name', 'email_address'] , axis=1)
df = pd.concat([df_train, df_test])
r, c = df.shape
df.head()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,NaN,-3504386.0,NaN,19794175.0,46950.0,18.0,42.0,4.0,NaN,1617011.0,174839.0,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,NaN,NaN,NaN,19250000.0,29336.0,108.0,88.0,30.0,NaN,1920000.0,22122.0,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,NaN,-4167.0,NaN,1624396.0,22884.0,39.0,13.0,14.0,NaN,NaN,1573324.0,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,NaN,NaN,NaN,NaN,118134.0,NaN,NaN,NaN,NaN,602671.0,907502.0,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,1250000.0,NaN,-262500.0,NaN,NaN,35818.0,144.0,199.0,25.0,NaN,375304.0,486.0,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [11]:
# 檢查 DataFrame 空缺值的狀態
def na_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    display(missing_data.head(c))
na_check(df)

,Missing Ratio
loan_advances,97.260274
director_fees,88.356164
restricted_stock_deferred,87.671233
deferral_payments,73.287671
deferred_income,66.438356
long_term_incentive,54.794521
bonus,43.835616
from_this_person_to_poi,41.095890
from_poi_to_this_person,41.095890
from_messages,41.095890


In [12]:
df["long_term_incentive"] = df["long_term_incentive"].fillna(df['long_term_incentive'].median())
df["bonus"] = df["bonus"].fillna(df['bonus'].median())
df["from_messages"] = df["from_messages"].fillna(df['from_messages'].median())
df["from_poi_to_this_person"] = df["from_poi_to_this_person"].fillna(df['from_poi_to_this_person'].median())
df["from_this_person_to_poi"] = df["from_this_person_to_poi"].fillna(df['from_this_person_to_poi'].median())
df["shared_receipt_with_poi"] = df["shared_receipt_with_poi"].fillna(df['shared_receipt_with_poi'].median())
df["to_messages"] = df["to_messages"].fillna(df['to_messages'].median())
df["other"] = df["other"].fillna(df['other'].median())
df["expenses"] = df["expenses"].fillna(df['expenses'].median())
df["salary"] = df["salary"].fillna(df['salary'].median())
df["exercised_stock_options"] = df["exercised_stock_options"].fillna(df['exercised_stock_options'].median())
df["restricted_stock"] = df["restricted_stock"].fillna(df['restricted_stock'].median())
df["total_payments"] = df["total_payments"].fillna(df['total_payments'].median())
df["total_stock_value"] = df["total_stock_value"].fillna(df['total_stock_value'].median())
df["director_fees"] = df["director_fees"].fillna(df['director_fees'].median())
df["loan_advances"] = df["loan_advances"].fillna(df['loan_advances'].median())
df["deferred_income"] = df["deferred_income"].fillna(df['deferred_income'].median())
df["deferral_payments"] = df["deferral_payments"].fillna(df['deferral_payments'].median())
df["restricted_stock_deferred"] = df["restricted_stock_deferred"].fillna(df['restricted_stock_deferred'].median())
df.head()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,227449.0,-3504386.0,108579.0,19794175.0,46950.0,18.0,42.0,4.0,41762500.0,1617011.0,174839.0,2748364.0,-146975.0,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,227449.0,-159792.0,108579.0,19250000.0,29336.0,108.0,88.0,30.0,41762500.0,1920000.0,22122.0,6843672.0,-146975.0,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,227449.0,-4167.0,108579.0,1624396.0,22884.0,39.0,13.0,14.0,41762500.0,442035.0,1573324.0,869220.0,-146975.0,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,227449.0,-159792.0,108579.0,1310813.5,118134.0,41.0,35.0,8.0,41762500.0,602671.0,907502.0,985032.0,-146975.0,224305.0,740.5,1211.0,2652612.0,985032.0
4,1250000.0,227449.0,-262500.0,108579.0,1310813.5,35818.0,144.0,199.0,25.0,41762500.0,375304.0,486.0,126027.0,-146975.0,240189.0,2188.0,2598.0,1639297.0,126027.0


In [13]:
# 將資料最大最小化
df = MinMaxScaler().fit_transform(df)

# 將前述轉換完畢資料 df , 重新切成 train_X, test_X
train_num = df_train.shape[0]
train_X = df[:train_num]
test_X = df[train_num:]
Y_train = train_data['poi']
x_train, x_test, y_train, y_test = train_test_split(train_X, Y_train, test_size = 0.2, random_state = 4)
# 使用三種模型 : 邏輯斯迴歸 / 梯度提升機 / 隨機森林, 參數使用 Random Search 尋找
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
lr = LogisticRegression(tol=0.001, penalty='l2', fit_intercept=True, C=1.0)
gdbt = GradientBoostingClassifier(tol=100, subsample=0.75, n_estimators=250,
                                  max_depth=10, learning_rate=0.03)
rf = RandomForestClassifier(n_estimators=100, criterion="gini", min_samples_split=2, min_samples_leaf=1, 
                            max_features="auto", max_depth=10, bootstrap=True)

In [14]:
# 線性迴歸預測檔 (結果有部分隨機, 請以 Kaggle 計算的得分為準, 以下模型同理)
lr.fit(x_train, y_train)
lr_pred = lr.predict(x_test)
Acc = accuracy_score(lr_pred, y_test)
print(Acc)
lr_sub = lr.predict_proba(test_X)
sub = pd.DataFrame(lr_sub, columns=['0','poi']).iloc[:,1:]
sub['name'] = test_features['name']
sub[['name','poi']].to_csv('mid_lr.csv', index = False)

0.8260869565217391


In [15]:
# 梯度提升機預測檔 
gdbt.fit(x_train, y_train)
gdbt_pred = gdbt.predict(x_test)
Acc = accuracy_score(gdbt_pred, y_test)
print(Acc)
gdbt_sub = gdbt.predict_proba(test_X)
sub = pd.DataFrame(gdbt_sub, columns=['0','poi']).iloc[:,1:]
sub['name'] = test_features['name']
sub[['name','poi']].to_csv('mid_gdbt.csv', index = False)

0.8695652173913043


In [16]:
# 隨機森林預測檔
rf.fit(x_train, y_train)
rf_pred = rf.predict(x_test)
Acc = accuracy_score(rf_pred, y_test)
print(Acc)
rf_sub = rf.predict_proba(test_X)
sub = pd.DataFrame(rf_sub, columns=['0','poi']).iloc[:,1:]
sub['name'] = test_features['name']
sub[['name','poi']].to_csv('mid_rf.csv', index = False)

0.8260869565217391


In [17]:
from mlxtend.classifier import StackingClassifier

meta_estimator = GradientBoostingClassifier(tol=10000, subsample=0.70, n_estimators=50, 
                                           max_features='auto', max_depth=10, learning_rate=0.3)

stacking = StackingClassifier(classifiers=[lr, gdbt, rf], meta_classifier=meta_estimator)

In [19]:
stacking.fit(x_train, y_train)
stacking_pred = stacking.predict(x_test)
Acc = accuracy_score(stacking_pred, y_test)
print(Acc)
stacking_pred = stacking.predict(test_X)
sub = pd.DataFrame(rf_sub, columns=['0','poi']).iloc[:,1:]
sub['name'] = test_features['name']
sub[['name','poi']].to_csv('mid.csv', index = False)

0.8260869565217391
